In [1]:
# for accessing file system
import os

# for numerical work
import numpy as np

# for loading netcdf files, for metadata
import xarray as xr

# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy,
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

# Used for processing netCDF time data
import requests

# Used for indexing via metadata
import pandas as pd

# local modules
from util import *

os.environ["VISUS_CACHE"] = "./visus_cache_can_be_erased"
os.environ["CURL_CA_BUNDLE"] = ""

# whether to download tiny netcdf or not
download = False

local_netcdf = "firesmoke_metadata.nc"

if download:
    # path to tiny NetCDF
    url = "https://github.com/sci-visus/NSDF-WIRED/raw/main/data/firesmoke_metadata.nc"

    # Download the file using requests
    response = requests.get(url)
    with open(local_netcdf, "wb") as f:
        f.write(response.content)

# open tiny netcdf with xarray and OpenVisus backend
ds = xr.open_dataset(local_netcdf, engine=OpenVisusBackendEntrypoint)

##### Process all metadata of file #####
### Lat/Lon ###
# get metadata to compute lon and lat
xorig = ds.XORIG
yorig = ds.YORIG
xcell = ds.XCELL
ycell = ds.YCELL
ncols = ds.NCOLS
nrows = ds.NROWS

longitude = np.linspace(xorig, xorig + xcell * (ncols - 1), ncols)
latitude = np.linspace(yorig, yorig + ycell * (nrows - 1), nrows)

# Create coordinates for lat and lon (credit: Aashish Panta)
ds.coords["lat"] = ("ROW", latitude)
ds.coords["lon"] = ("COL", longitude)

# Replace col and row dimensions with newly calculated lon and lat arrays (credit: Aashish Panta)
ds = ds.swap_dims({"COL": "lon", "ROW": "lat"})

### Timestamps ###
# get all tflags
tflag_values = ds["TFLAG"].values

# to store pandas timestamps
timestamps = []

# convert all tflags to pandas timestamps, store in timestamps list
for tflag in tflag_values:
    timestamps.append(pd.Timestamp(parse_tflag(tflag[0])))

# set coordinates to each timestep with these pandas timestamps
ds.coords["time"] = ("time", timestamps)

##### Functions for bokeh to access data #####
def get_latslons():
    """
    Return a numpy array of all lats and lons used in dataset in mercator coordinates
    """
    # get list of latitudes and longitudes
    data_stacked_index = ds["PM25"][0].stack(lat_lon=["lat", "lon"])

    # numpy array to return
    mercator_latlons = np.zeros((np.shape(data_stacked_index)[1], 2))
    for i, tup in enumerate(latlon_to_mercator_iter(data_stacked_index.lat_lon.data)):
        mercator_latlons[i][0] = tup[0]
        mercator_latlons[i][1] = tup[1]

    return mercator_latlons


def get_pm25(date, hour, res):
    """
    Return a numpy array of pm2.5 values for all hours of given date at specified resolution
    :param string date: Date to query data for, in format "YYYY-MM-DD"
    :param int hour: Hour to query data for, from hours 0-23
    :param int res: IDX resolution to use
    """
    # make timestamp
    print(date)
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[-2:])
    t = get_pd_timestamp(year, month, day, hour)

    # get slice, then flatten it
    # TODO: need to handle when a date that is unavailable is given, e.g. 2021-03-03
    data_array_at_time = ds["PM25"].loc[t, :, :, res].data[:, :, 0]

    # TODO: is this flattened right?
    return data_array_at_time.flatten()


ov.LoadDataset(http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_smoke_BSC&cached=1)
PM25
Adding field  PM25 shape  [27357, 381, 1081, 21] dtype  float32 labels  ['time', 'ROW', 'COL', 'resolution'] Max Resolution  20


In [2]:
ds

<xarray.Dataset> Size: 946GB
Dimensions:            (time: 27357, lat: 381, lon: 1081, resolution: 21,
                        VAR: 1, DATE-TIME: 2)
Coordinates:
  * lat                (lat) float64 3kB 32.0 32.1 32.2 32.3 ... 69.8 69.9 70.0
  * lon                (lon) float64 9kB -160.0 -159.9 -159.8 ... -52.1 -52.0
  * time               (time) datetime64[ns] 219kB 2021-03-04 ... 2024-06-27T...
Dimensions without coordinates: resolution, VAR, DATE-TIME
Data variables:
    PM25               (time, lat, lon, resolution) float32 946GB ...
    TFLAG              (time, VAR, DATE-TIME) int32 219kB 2021063 0 ... 230000
    wrf_arw_init_time  (time, VAR, DATE-TIME) int32 219kB ...
    resampled          (time) bool 27kB ...
    CDATE              (time) int32 109kB ...
    CTIME              (time) int32 109kB ...
    WDATE              (time) int32 109kB ...
    WTIME              (time) int32 109kB ...
    SDATE              (time) int32 109kB ...
    STIME              (time) int32 109kB ...
Attributes: (12/28)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    TSTEP:          10000
    NTHIK:          1
    NCOLS:          1081
    ...             ...
    GDNAM:          HYSPLIT CONC    
    UPNAM:          hysplit2netCDF  
    VAR-LIST:       PM25            
    FILEDESC:       Hysplit Concentration Model Output                       ...
    HISTORY:        
    idx_url:        http://atlantis.sci.utah.edu/mod_visus?dataset=UBC_fire_s...

In [3]:
get_latslons()

Using Max Resolution:  20
Time: 0, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25


array([[-17811118.52692377,   3763310.62714465],
       [-17799986.57767856,   3763310.62714465],
       [-17788854.62843336,   3763310.62714465],
       ...,
       [ -5810877.24059135,  11068715.8436786 ],
       [ -5799745.29134615,  11068715.8436786 ],
       [ -5788613.34210094,  11068715.8436786 ]])

In [4]:
# make timestamp
date = "2021-03-04"
year = int(date[0:4])
month = int(date[5:7])
day = int(date[-2:])
hour = 0
res = 0

t = get_pd_timestamp(year, month, day, hour)
arr = get_pm25(date, hour, res)
# # get slice, then flatten it
# # TODO: need to handle when a date that is unavailable is given, e.g. 2021-03-03
# data_array_at_time = ds["PM25"].loc[t, :, :, res]

2021-03-04
Using Max Resolution:  20
Time: 0, max_resolution: 20, logic_box=(0, 1081, 0, 381), field: PM25


In [7]:
np.shape(arr.flatten())

(411861,)